In [30]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gurobipy as gp
from gurobipy import GRB

## Task 1] Build the optimisation model

In [31]:
# Load the data from the data folder
wind_forecast = pd.read_csv('../Data/1.Wind forecast profile.csv', delimiter=';') # Made up data
load = pd.read_csv('../Data/1.Load profile.csv', delimiter=';')
bus = pd.read_csv('../Data/B (power transfer factor of each bus to each line).csv', delimiter=';')
max_prod = pd.read_csv('../Data/Maximum production of generating units.csv', delimiter=';')
min_prod = pd.read_csv('../Data/Minimum production of generating units.csv', delimiter=';')
min_down_time = pd.read_csv('../Data/Minimum down time of generating units.csv', delimiter=';')
min_up_time = pd.read_csv('../Data/Minimum up time of generating units.csv', delimiter=';')
prod_cost = pd.read_csv('../Data/Production cost of generating units.csv', delimiter=';')
ramp_rate = pd.read_csv('../Data/Ramping rate of generating units.csv', delimiter=';')
start_up_cost = pd.read_csv('../Data/Start-up cost of generating units.csv', delimiter=';')
transmission_cap = pd.read_csv('../Data/Transmission capacity of lines.csv', delimiter=';')
samples = pd.read_csv('../Data/sample_simon.csv', delimiter=',')
# Overwrite the load DataFrame with selected columns from samples
load = samples[['Sample_Nr', 'Hour', 'L1', 'L2', 'L3']]

# Overwrite the wind_forecast DataFrame with selected columns from samples
wind_forecast = samples[['Sample_Nr', 'Hour', 'W1', 'W2']]


In [32]:
Nodes = ['Node 1', 'Node 2', 'Node 3', 'Node 4', 'Node 5', 'Node 6']
Generator = ['G1', 'G2', 'G3']
Generator_node = {'Node 1': 'G1', 'Node 2': 'G2', 'Node 6': 'G3'}
Load = ['L1', 'L2', 'L3']
Load_node = {'Node 4': 'L1', 'Node 5': 'L2', 'Node 6': 'L3'}
Wind = ['W1', 'W2']
Wind_node = {'Node 4': 'W1', 'Node 5': 'W2'}

In [33]:
# Create matrix with the nodes as columns and the generators, loads and winds as rows, with 1 if connected to the node
Gen_n = np.zeros((len(Generator), len(Nodes)))
Load_n = np.zeros((len(Load), len(Nodes)))
Wind_n = np.zeros((len(Wind), len(Nodes)))

# Populate the matrix
for i, g in enumerate(Generator):  # Iterate over generators
    for j, node in enumerate(Nodes):  # Iterate over nodes
        if Generator_node.get(node) == g:  # Check if generator is connected to the node
            Gen_n[i, j] = 1

for i, l in enumerate(Load):  # Iterate over loads
    for j, node in enumerate(Nodes):  # Iterate over nodes
        if Load_node.get(node) == l:  # Check if load is connected to the node
            Load_n[i, j] = 1

for i, w in enumerate(Wind):  # Iterate over winds
    for j, node in enumerate(Nodes):  # Iterate over nodes
        if Wind_node.get(node) == w:  # Check if wind is connected to the node
            Wind_n[i, j] = 1
            


In [34]:
# Define the input data class
class InputData:
    
    def __init__(
        self,
        wind_forecast: pd.DataFrame, 
        bus: pd.DataFrame,
        load: pd.DataFrame,
        max_prod: pd.DataFrame,
        min_prod: pd.DataFrame,
        min_down_time: pd.DataFrame,
        min_up_time: pd.DataFrame,
        prod_cost: pd.DataFrame,
        ramp_rate: pd.DataFrame,
        start_up_cost: pd.DataFrame,
        transmission_cap: pd.DataFrame
    ):
        self.time = list(range(len(wind_forecast)))
        self.wind_forecast = wind_forecast
        self.bus = bus
        self.load = load
        self.max_prod = max_prod
        self.min_prod = min_prod
        self.min_down_time = min_down_time
        self.min_up_time = min_up_time
        self.prod_cost = prod_cost
        self.ramp_rate = ramp_rate
        self.start_up_cost = start_up_cost
        self.transmission_cap = transmission_cap
        self.M = 1000000  # Penalty for having flexible demand
        self.Gen_n = Gen_n  # Matrix mapping generators to nodes
        self.Load_n = Load_n # Matrix mapping loads to nodes
        self.Wind_n = Wind_n # Matrix mapping wind to nodes
        
        


In [35]:
class Expando(object):
    '''
        A small class which can have attributes set
    '''
    pass

In [36]:
# Define the optimization model class

class EconomicDispatch():
        
        def __init__(self, input_data: InputData):
            self.data = input_data 
            self.variables = Expando()
            self.constraints = Expando() 
            self.results = Expando() 
            self._build_model() 
            
        def _build_variables(self):
            # one binary variable for the status of each generator
            self.variables.status = {
                (i, t): self.model.addVar(vtype=GRB.BINARY, 
                                            name='status_G{}_{}'.format(i, t)) 
                                            for i in range(1, len(self.data.max_prod)+1) 
                                            for t in self.data.time}
            
            # one variable for each generator for each time of the day
            self.variables.prod_gen = {
                 (i, t): self.model.addVar(lb=0, ub=self.data.max_prod.iloc[i-1, 0], 
                                           name='generation_G{}_{}'.format(i, t)) 
                                           for i in range(1, len(self.data.max_prod)+1) 
                                           for t in self.data.time}
            
            # one variable for each wind generator for each time of the day
            self.variables.prod_wind = {
                 (i, t): self.model.addVar(lb=0, ub=self.data.wind_forecast.iloc[t, i], 
                                            name='wind_generation_W{}_{}'.format(i, t)) 
                                            for i in range(1, len(self.data.wind_forecast.iloc[0, :])) 
                                            for t in self.data.time}
            
            # one variable for each start-up cost for each generator
            self.variables.start_up_cost = {
                 (i, t): self.model.addVar(lb=0, 
                                            name='start_up_cost_G{}_{}'.format(i, t)) 
                                            for i in range(1, len(self.data.max_prod)+1) 
                                            for t in self.data.time}
            
            # add two slack variables to always make the model feasible, allowing the demand to be flexible
            self.variables.epsilon = {
                 (n, t): self.model.addVar(lb=0, 
                                           name='epsilon_Bus{}_{}'.format(n, t)) 
                                           for n in range(1, len(self.data.bus.iloc[0,:])+1) 
                                           for t in self.data.time}
            self.variables.delta = {
                 (n, t): self.model.addVar(lb=0, 
                                           name='delta_Bus{}_{}'.format(n, t))
                                           for n in range(1, len(self.data.bus.iloc[0,:])+1)
                                           for t in self.data.time}
            
            
        def _build_constraints(self):
            # Minimum capacity of the generator
            self.constraints.min_capacity = {
                (i, t): self.model.addConstr(
                    self.variables.prod_gen[i, t] >= self.data.min_prod.iloc[i-1, 0] * self.variables.status[i, t]
                ) for i in range(1, len(self.data.max_prod)+1) for t in self.data.time}
            # Maximum capacity of the generator
            self.constraints.max_capacity = {
                (i, t): self.model.addConstr(
                    self.variables.prod_gen[i, t] <= self.data.max_prod.iloc[i-1, 0] * self.variables.status[i, t]
                ) for i in range(1, len(self.data.max_prod)+1) for t in self.data.time}

            # Power balance constraint
            self.constraints.power_balance = {
                t: self.model.addConstr(
                    gp.quicksum(self.variables.prod_gen[i, t] for i in range(1, len(self.data.max_prod)+1)) + 
                    gp.quicksum(self.variables.prod_wind[i, t] for i in range(1, len(self.data.wind_forecast.iloc[0, :]))) == 
                    gp.quicksum(self.data.load.iloc[t, i] * Load_n[i-1, n-1] + self.variables.epsilon[n, t] - self.variables.delta[n, t] 
                        for i in range(1, len(self.data.load.iloc[0, :]))
                        for n in range(1, len(self.data.bus.iloc[0,:])+1))
                ) for t in self.data.time}

            # Transmission capacity constraint
            self.constraints.transmission_capacity_up = {
                    (l, t): self.model.addConstr(
                        gp.quicksum(self.data.bus.iloc[l-1, n-1] *
                                (self.variables.prod_gen[i, t] * Gen_n[i-1, n-1] +
                                self.variables.prod_wind[i, t] * Wind_n[i-1, n-1] -
                                self.variables.epsilon[n, t] +
                                self.variables.delta[n, t]) for n in range(1, len(self.data.bus.iloc[0,:])+1)) <=
                        self.data.transmission_cap.iloc[l-1, 0]
                    ) for l in range(1, len(self.data.transmission_cap)+1)
                    for t in self.data.time}
            self.constraints.transmission_capacity_down = {
                    (l, t): self.model.addConstr(
                        gp.quicksum(self.data.bus.iloc[l-1, n-1] *
                                (self.variables.prod_gen[i, t] * Gen_n[i-1, n-1] +
                                self.variables.prod_wind[i, t] * Wind_n[i-1, n-1] -
                                self.variables.epsilon[n, t] +
                                self.variables.delta[n, t]) for n in range(1, len(self.data.bus.iloc[0,:])+1)) >=
                        -self.data.transmission_cap.iloc[l-1, 0]
                    ) for l in range(1, len(self.data.transmission_cap)+1)
                    for t in self.data.time}
                                     




            #Start-up costs constraint
            self.constraints.start_up_cost = {
                (i, t): self.model.addConstr(
                    self.variables.start_up_cost[i, t] >= self.data.start_up_cost.iloc[i-1, 0] * (self.variables.status[i, t] - self.variables.status[i, t-1])
                ) for i in range(1, len(self.data.max_prod)+1) for t in self.data.time if t > 0}
            self.constraints.start_up_cost_0 = {
                i: self.model.addConstr(
                    self.variables.start_up_cost[i, 0] >= self.data.start_up_cost.iloc[i-1, 0] * self.variables.status[i, 0]
                ) for i in range(1, len(self.data.max_prod)+1)}
            
            # Ramping constraint
            self.constraints.ramping_up = {
                (i, t): self.model.addConstr(
                    self.variables.prod_gen[i, t] - self.variables.prod_gen[i, t-1] <= self.data.ramp_rate.iloc[i-1, 0]
                ) for i in range(1, len(self.data.max_prod)+1) for t in self.data.time if t > 0}
            self.constraints.ramping_down = {
                (i, t): self.model.addConstr(
                    self.variables.prod_gen[i, t-1] - self.variables.prod_gen[i, t] <= self.data.ramp_rate.iloc[i-1, 0]
                ) for i in range(1, len(self.data.max_prod)+1) for t in self.data.time if t > 0}
            
            # Minimum up time constraint
            self.constraints.min_up_time = {
                (i, t, to): self.model.addConstr(
                    -self.variables.status[i, t - 1] + self.variables.status[i, t] - self.variables.status[i, to] <= 0
                ) for i in range(1, len(self.data.max_prod)+1) 
                for t in self.data.time 
                for to in range(t, min(t + self.data.min_up_time.iloc[i-1, 0], len(self.data.time))) if t > 0}
            
            # Minimum down time constraint
            self.constraints.min_down_time = {
                (i, t, to): self.model.addConstr(
                    self.variables.status[i, t - 1] - self.variables.status[i, t] + self.variables.status[i, to] <= 1
                ) for i in range(1, len(self.data.max_prod)+1) 
                for t in self.data.time 
                for to in range(t, min(t + self.data.min_down_time.iloc[i-1, 0], len(self.data.time))) if t > 0}
            


        def _build_objective(self):
            # Objective function
            self.model.setObjective(
                gp.quicksum(self.data.prod_cost.iloc[i-1, 0]*self.variables.prod_gen[i, t] for i in range(1, len(self.data.max_prod)+1) for t in self.data.time) +
                gp.quicksum(self.variables.start_up_cost[i, t] for i in range(1, len(self.data.max_prod)+1) for t in self.data.time) +
                self.data.M * (gp.quicksum(self.variables.epsilon[n, t] + self.variables.delta[n, t] for n in range(1, len(self.data.bus.iloc[0,:])+1) for t in self.data.time))
            )

        def _build_model(self):
            self.model = gp.Model('EconomicDispatch')
            self._build_variables()
            self._build_constraints()
            self._build_objective()
            self.model.update()

        def optimize(self):
            self.model.optimize()
            self._extract_results()

        def _extract_results(self):
            self.results.production = pd.DataFrame({
                #'time': [t for t in self.data.time],
                #'status G1': [self.variables.status[1, t].x for t in self.data.time],
                #'status G2': [self.variables.status[2, t].x for t in self.data.time],
                #'status G3': [self.variables.status[3, t].x for t in self.data.time],
                #'start_up_cost 1': [self.variables.start_up_cost[1, t].x for t in self.data.time],
                #'start_up_cost 2': [self.variables.start_up_cost[2, t].x for t in self.data.time],
                #'start_up_cost 3': [self.variables.start_up_cost[3, t].x for t in self.data.time],
                'generation 1': [self.variables.prod_gen[1, t].x for t in self.data.time],
                'generation 2': [self.variables.prod_gen[2, t].x for t in self.data.time],
                'generation 3': [self.variables.prod_gen[3, t].x for t in self.data.time],
                'wind generation 1': [self.variables.prod_wind[1, t].x for t in self.data.time],
                'wind generation 2': [self.variables.prod_wind[2, t].x for t in self.data.time],
                'load 1': [self.data.load.iloc[t, 1] for t in self.data.time],
                'load 2': [self.data.load.iloc[t, 2] for t in self.data.time],
                'load 3': [self.data.load.iloc[t, 3] for t in self.data.time],
                'epsilon 1': [self.variables.epsilon[1, t].x for t in self.data.time],
                'delta 1': [self.variables.delta[1, t].x for t in self.data.time],
                'epsilon 2': [self.variables.epsilon[2, t].x for t in self.data.time],
                'delta 2': [self.variables.delta[2, t].x for t in self.data.time],
                'epsilon 3': [self.variables.epsilon[3, t].x for t in self.data.time],
                'delta 3': [self.variables.delta[3, t].x for t in self.data.time],
                'epsilon 4': [self.variables.epsilon[4, t].x for t in self.data.time],
                'delta 4': [self.variables.delta[4, t].x for t in self.data.time],
                'epsilon 5': [self.variables.epsilon[5, t].x for t in self.data.time],
                'delta 5': [self.variables.delta[5, t].x for t in self.data.time],
                'epsilon 6': [self.variables.epsilon[6, t].x for t in self.data.time],
                'delta 6': [self.variables.delta[6, t].x for t in self.data.time]
            })
            self.results.unit_commitment = pd.DataFrame({
                'time': [t for t in self.data.time],
                'G1': [self.variables.status[1, t].x for t in self.data.time],
                'G2': [self.variables.status[2, t].x for t in self.data.time],
                'G3': [self.variables.status[3, t].x for t in self.data.time]
            })
            

In [37]:
# Run the model
input_data = InputData(wind_forecast, bus, load, max_prod, min_prod, min_down_time, min_up_time, prod_cost, ramp_rate, start_up_cost, transmission_cap)
model = EconomicDispatch(input_data)
model.optimize()
model.results.production


IndexError: index 3 is out of bounds for axis 0 with size 3

In [ ]:
model.results.unit_commitment #to be used for the next steps

In [ ]:
# Group the data by 'Sample_Number'
grouped_samples = samples.groupby('Sample_Number')


In [ ]:
for sample_number, group in grouped_samples:
    if len(group) != 24:
        print(f"Sample {sample_number} does not have 24 rows. It has {len(group)} rows.")


In [ ]:
    # Ensure the number of hours is correct (24 rows for each sample)
    if len(wind_forecast_sample) != 24:
        print(f"Sample {sample_number} does not have exactly 24 rows. Found: {len(wind_forecast_sample)}")
 

In [ ]:
print(f"Wind Forecast Sample for Sample {sample_number}:\n{wind_forecast_sample}")
print(f"Load Sample for Sample {sample_number}:\n{load_sample}")
print(f"Status Variables Dimensions: {len(model.variables.status)}")


In [ ]:
# Group the data by 'Sample_Number'
grouped_samples = samples.groupby('Sample_Number')

# Prepare storage for results
all_results = []

# Iterate over each sample group
for sample_number, group in grouped_samples:
    print(f"Processing Sample {sample_number}...")

    # Extract 24-hour data for the current sample
    wind_forecast_sample = group[['Hour', 'Wind_W1', 'Wind_W2']].reset_index(drop=True)
    load_sample = group[['Hour', 'Load_L1', 'Load_L2', 'Load_L3']].reset_index(drop=True)

    # Ensure the number of hours is correct (24 rows for each sample)
    if len(wind_forecast_sample) != 24:
        print(f"Sample {sample_number} does not have exactly 24 rows. Found: {len(wind_forecast_sample)}")
        continue

    # Create InputData for the current sample
    input_data = InputData(
        wind_forecast=wind_forecast_sample,
        bus=bus,
        load=load_sample,
        max_prod=max_prod,
        min_prod=min_prod,
        min_down_time=min_down_time,
        min_up_time=min_up_time,
        prod_cost=prod_cost,
        ramp_rate=ramp_rate,
        start_up_cost=start_up_cost,
        transmission_cap=transmission_cap
    )

    # Run the optimization model
    model = EconomicDispatch(input_data)
    model.optimize()

    # Extract results for each hour in the sample
    for hour_index in range(len(wind_forecast_sample)):  # Use length of the sample for iteration
        try:
            result = {
                'Sample_Number': sample_number,
                'Hour': hour_index + 1,  # Convert 0-based index to 1-based for consistency
                'G1_Status': model.variables.status[1, hour_index].x,
                'G2_Status': model.variables.status[2, hour_index].x,
                'G3_Status': model.variables.status[3, hour_index].x
            }
            all_results.append(result)
        except IndexError as e:
            print(f"Error at Sample {sample_number}, Hour {hour_index + 1}: {e}")
            break  # Stop processing this sample to avoid further errors

# Convert all results to a DataFrame
results_df = pd.DataFrame(all_results)

# Print or save results
print(results_df.head())


In [ ]:
# Combine all results into a DataFrame
results_df = pd.DataFrame(all_results)

# Display the first few rows of the results
print(results_df.head())


In [ ]:
# Merge results with original samples
full_dataset = samples.merge(results_df, on=['Sample_Number', 'Hour'])

# Define features and labels
features = full_dataset[['Load_L1', 'Load_L2', 'Load_L3', 'Wind_W1', 'Wind_W2']]
labels = full_dataset[['G1_Status', 'G2_Status', 'G3_Status']]

# Confirm dataset structure
print(features.head())
print(labels.head())


In [ ]:
##Classification

In [ ]:
features = model.results.production[['generation 1', 'generation 2', 'generation 3', 
                                            'wind generation 1', 'wind generation 2',
                                            'load 1', 'load 2', 'load 3']]

# Use the unit commitment results as labels
labels = model.results.unit_commitment[['G1', 'G2', 'G3']]

# Merge features and labels on the 'time' column
dataset = pd.concat([features, labels], axis=1)

# Separate features (X) and labels (y)
X = dataset.drop(columns=['G1', 'G2', 'G3'])
y = dataset[['G1', 'G2', 'G3']]



In [ ]:
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Training Set: {X_train.shape}, Validation Set: {X_val.shape}, Test Set: {X_test.shape}")


In [ ]:
# Check if units are always ON or OFF
print(y_train.mean())  # If mean = 1 or 0, the unit is always ON or OFF

# Filter out units that are always ON or OFF
units_to_classify = y_train.columns[(y_train.mean() > 0) & (y_train.mean() < 1)]
print(f"Units to classify: {units_to_classify}")


In [ ]:
# Linear Classifier

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Define logistic regression models for each generator
logistic_models = {}

# Loop through each generator in the labels
for unit in y.columns:  # e.g., 'G1', 'G2', 'G3'
    print(f"\nTraining Logistic Regression for {unit}...")

    # Extract training and validation data for this unit
    y_train_unit = y_train[unit]
    y_val_unit = y_val[unit]

    # Initialize and train the logistic regression model
    logistic = LogisticRegression(random_state=42)
    logistic.fit(X_train, y_train_unit)
    logistic_models[unit] = logistic

    # Predict on validation set
    y_pred_val = logistic.predict(X_val)

    # Evaluate performance
    accuracy = accuracy_score(y_val_unit, y_pred_val)
    print(f"Validation Accuracy for {unit}: {accuracy}")
    print(classification_report(y_val_unit, y_pred_val))



In [ ]:
## Non-Linear Classifier

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

# Define SVM models for each generator
svm_models = {}

# Loop through each generator in the labels
for unit in y.columns:  # e.g., 'G1', 'G2', 'G3'
    print(f"\nTraining SVM (RBF Kernel) for {unit}...")

    # Extract training and validation data for this unit
    y_train_unit = y_train[unit]
    y_val_unit = y_val[unit]

    # Initialize and train the SVM model
    svm = SVC(kernel='rbf', random_state=42)
    svm.fit(X_train, y_train_unit)
    svm_models[unit] = svm

    # Predict on validation set
    y_pred_val = svm.predict(X_val)

    # Evaluate performance
    accuracy = accuracy_score(y_val_unit, y_pred_val)
    print(f"Validation Accuracy for {unit}: {accuracy}")
    print(classification_report(y_val_unit, y_pred_v))


In [ ]:
#Evaluate Models

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Evaluate all models on the test set
for unit in y_test.columns:  # e.g., 'G1', 'G2', 'G3'
    print(f"\nEvaluating models for {unit}:")

    # Logistic Regression
    logistic_model = logistic_models[unit]  # Retrieve the trained logistic regression model
    y_pred_test_logistic = logistic_model.predict(X_test)  # Predict on the test set
    print(f"Logistic Regression Test Accuracy for {unit}: {accuracy_score(y_test[unit], y_pred_test_logistic)}")
    print(classification_report(y_test[unit], y_pred_test_logistic))

    # SVM
    svm_model = svm_models[unit]  # Retrieve the trained SVM model
    y_pred_test_svm = svm_model.predict(X_test)  # Predict on the test set
    print(f"SVM Test Accuracy for {unit}: {accuracy_score(y_test[unit], y_pred_test_svm)}")
    print(classification_report(y_test[unit], y_pred_test_svm))

